In [1]:
!pip install opencv-python
!pip install pypylon
!pip install pypylon-opencv-viewer
!pip install pytesseract
!pip install natsort

In [2]:
import argparse
import itertools
import os
import shutil
import re
import copy
import socket
import threading
import queue
import subprocess

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import NoNorm
from matplotlib.colors import Normalize
import cv2

from pypylon import pylon 
from pypylon_opencv_viewer import BaslerOpenCVViewer

from IPython.display import clear_output, display

import pytesseract
from PIL import Image

from collections import OrderedDict
import math

import yaml
from natsort import natsorted
from scipy.spatial import distance

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

%matplotlib inline

In [3]:
def connect_camera(serial_number):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    Returns
    -------
    camera : object
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
        return camera
    else:
        return None    

In [4]:
def load_image(file_path):
    assert os.path.exists(file_path), 'File does NOT exist! (' + file_path + ')'
    return cv2.imread(file_path)

def save_image(image, file_path):
    return cv2.imwrite(file_path, image)

In [5]:
def show_camera_window(*imgs, scale=1):
    def print_xy(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            print('x = %d, y = %d'% (x, y))  
        
    for i, img in enumerate(imgs, 1):
        window_name_id = 'Camera capture' + ' ' + str(i)
        
        h,w = img.shape[:2]
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, int(w * scale), int(h * scale))
        cv2.setMouseCallback(window_name_id, print_xy)
        if len(imgs) > 1:
            cv2.moveWindow(window_name_id, (i-1)*int(w * scale), 0)
        cv2.imshow(window_name_id, img)

In [6]:
def to_gray(img_bgr):
    ''' Converts image to monochrome
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    if len(img_bgr.shape) == 2:
        return img_bgr
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

def to_hsv(img_bgr):
    ''' Converts image to HSV (hue, saturation, value) color space.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    dst = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    return dst

def to_rgb(img_bgr):
    ''' Converts image to RGB (red, green, blue) color space from BGR.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    dst = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return dst

def negative(img):
    ''' Converts image to its negative.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    dst = 255 - img
    return dst

def normalize(img):
    '''Normalizes image using min-max normalization from its values to values 0 - 255.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    '''
    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

def crop(img, tl_x, tl_y, br_x, br_y):
    ''' Crops image by added coordinates.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    tl_x : int
        TOP-LEFT corner's x-coordinate
    tl_y : int
        TOP-LEFT corner's y-coordinate
    br_x : int
        BOTTOM-RIGHT corner's x-coordinate
    br_y : int
        BOTTOM-RIGHT corner's y-coordinate
    Returns
    -------
    Output image.
    '''
    roi = img[tl_y:br_y, tl_x:br_x]
    return roi    

def crop_by_bounding_rect(img_bin):
    ''' Crops binary image by ONE bounding rectangle corresponding to ALL objects in the binary image.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    Returns
    -------
    Output cropped image.
    '''
    assert len(img_bin.shape) == 2, 'Input image is NOT binary!'
    
    contours, _  = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tl_x, tl_y, w, h = cv2.boundingRect(contours[0])
    return crop(img_bin, tl_x, tl_y, tl_x+w, tl_y+h)

def crop_contour(contour, image):
    ''' Crops contour in respect to its bounding rectangle. 
    It's the fastest method, but could include other parts 
    of image than just contour if the contour is irregulary shaped.
    
    Parameters
    ----------
    contour : numpy.ndarray
        Contour that represents the area from image to be cropped. 
        The bounding rectangle of contour is used.
    img_bin : numpy.ndarray
        Input binary image.
    Returns
    -------
    Output cropped image.
    '''
    x,y,w,h = cv2.boundingRect(contour)
    return image[y:y+h, x:x+w]

def contour_to_image(contour, image, size=None):
    ''' Creates new image from the contour. 
    It's similar to contour cropping but it's not that fast. 
    It does not suffer from the known error if the contour is irregulary shaped.
    
    Parameters
    ----------
    contour : numpy.ndarray
        Contour that represents the area from image to be cropped. 
    img_bin : numpy.ndarray
        Input binary image.
    size : tuple
        Optional size of the created image. 
        If it's not used, the image's size is the same as the 
        size of bounding rectangle of the input contour.
    Returns
    -------
    Output cropped image.
    '''
    if size is None:
        _, _, w, h = cv2.boundingRect(contour)
        size = (w, h)

    assert type(size) is tuple, 'Param size should be a tuple!'
    blank = np.zeros_like(image)
    half_x = int(size[0] * 0.5)
    half_y = int(size[1] * 0.5)

    c = get_center(contour)
    cv2.drawContours(blank, [contour], -1, (255, 255, 255), cv2.FILLED)

    return blank[c[1]-half_y:c[1]+half_y, c[0]-half_x:c[0]+half_x].copy()

def resize(image, size, method=cv2.INTER_AREA):
    ''' Resizes the image to the preffered size.  
    Method of resizing is well suited for making the images smaller rather than larger
    (cv2.INTER_AREA). For making images larger, use other cv2.INTER_### instead.
    
    Parameters
    ----------
    image : numpy.ndarray
        Contour that represents the area from image to be cropped. 
    size : tuple
        New size of the resized image. 
    method : int
        Optional argument. For more information see cv2.INTER_### parameters.
    Returns
    -------
    Output resized image.
    '''
    assert type(size) is tuple, 'Variable size is NOT a tuple!'
    return cv2.resize(image, size, method)

In [7]:
def rotated_rectangle(image, idx):
    ''' Draws rotated rectangle into the image from indexes of binary image. 
    You can get the indexes of objects from binary image using cv2.findNonZero().
    Input image is not modified.
    '''
    res = image.copy()
    rect = cv2.minAreaRect(idx)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(res, [box], -1, (255, 255, 255), 1)
    return res, rect

def get_center(contour):
    ''' Gets the center of contour in pixels in tuple format.
    
    Parameters
    ----------
    contour : numpy.ndarray
        input contour.
    Returns
    -------
    Center in pixels in tuple format.
    '''
    M = cv2.moments(contour)
    cX = int(M['m10'] / M['m00'])
    cY = int(M['m01'] / M['m00'])
    
    return (cX, cY)

In [8]:
def segmentation_one_threshold(img, threshold):
    '''Segments image into black & white using one threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    threshold : int
        Pixels with value lower than threshold are considered black, the others white.
    Returns
    -------
    Output image.
    '''
    _, dst = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return dst

def segmentation_auto_threshold(img):
    '''Segments image into black & white using automatic threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    _, dst = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst

def segmentation_two_thresholds(img, lower, higher):
    '''Segments image into black & white using two thresholds
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    lower : int
        Pixels with value lower than threshold are considered black, the others white.
    higher : int
        Pixels with value higher than threshold are considered black, the others white.
    Returns
    -------
    Output image.
    '''
    return cv2.inRange(img, min(lower, higher), max(lower, higher))

def segmentation_adaptive_threshold(img, size, constant=0):
    '''Segments image into black & white using calculated adaptive 
    threshold using Gaussian function in pixel neighbourhood.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    size : int
        Size of used gaussian. Lowest value is 3. Algorithm uses only odd numbers.
    constant : int
        Value that is added to calculated threshlod. It could be negative as well as zero as well as positive number.
    Returns
    -------
    Output binary image.
    '''
    if size < 3:
        size = 3
    elif size % 2 == 0:
        size -= 1
    return cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, size, int(constant))

def apply_mask(img, mask_bin):
    '''Masks colored image with binary mask. Output image is just logical AND between two images.'''
    return cv2.bitwise_and(img, img, mask = mask_bin)

def find_contours(img_bin, min_area=0, max_area=1000000, fill=True, external=True):
    '''Finds contours in binary image and filters them using their area. Then it draws binary image
    from filtered contours. It counts contours as well.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    min_area : int
        Size of contour that is used to filter all smaller contours out.
    max_area : int
        Size of contour that is used to filter all larger contours out.
    Returns
    -------
    contour_drawn : numpy.ndarray
        Output binary image with drawn filled filtered contours.
    count : int
        Number of found and filtered contours.
    contours : list
        Found contours.
    '''
    mode = cv2.RETR_EXTERNAL
    if not external:
        mode = cv2.RETR_LIST
    contours, _  = cv2.findContours(img_bin, mode, cv2.CHAIN_APPROX_SIMPLE)
    contours =  [c for c in contours if cv2.contourArea(c) > min_area and cv2.contourArea(c) < max_area]
    thick = cv2.FILLED
    if not fill: thick = 2
    contour_drawn = cv2.drawContours(np.zeros(img_bin.shape, dtype=np.uint8), contours, -1, color=(255, 255, 255), thickness=thick)
    return contour_drawn, len(contours), contours

In [9]:
def filtration_box(img, filter_size):
    '''Filters image noise using box blur algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of box blur filter.
    Returns
    -------
    Output image.
    '''
    return cv2.blur(img, (filter_size, filter_size))

def filtration_median(img, filter_size):
    '''Filters image noise using median algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of median filter.
    Returns
    -------
    Output image.
    '''
    return cv2.medianBlur(img, filter_size)   

def filtration_gauss(img, filter_size, sigma_x):
    '''Filters image noise using Gaussian blur algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of Gaussian filter.
    Returns
    -------
    Output image.
    '''
    return cv2.GaussianBlur(img, (filter_size, filter_size), sigma_x) 

def fill_holes(img_bin, close=False, size=5):
    '''Fill holes in found contours. It could merge the contour using close input with appropriate size.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    close : boolean
        If it should merge contours with missing points using close operation.
    size : int
        Size of close operation element.
    Returns
    -------
    Output binary image.
    '''
    if close:
        struct = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
        img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, struct)
    res, _, _ = find_contours(img_bin)
    return res

In [10]:
def apply_fft(image):
    ''' Applies FFT on image given.
    
    Parameters
    ----------
    image : 2D array
        Image to perform FFT on.
    Returns
    -------
    mag_spec : 2D array
        Normalized magnitude spectrum.
    fftcls_shift : 2D array
        Centered product of FFT.
    '''
    fftcls = np.fft.fft2(image)
    fftcls_shift = np.fft.fftshift(fftcls)
    mag_spec = 20*np.log(np.abs(fftcls_shift))
    return cv2.normalize(mag_spec,  None, 0, 255, cv2.NORM_MINMAX,cv2.CV_8U), fftcls_shift

def inverse_fft(fft_shift, filter_mask=None):
    ''' Applies inverse FFT.
    
    Parameters
    ----------
    fft_shift : 2D array
        Shifted computed FFT
    filter_mask : 2D array
        2D array mask containing 255 and 0 values.
    Returns
    -------
    img_back : 2D array
        Image made by inverse FFT.
    '''
    fftshift = np.copy(fft_shift)
    if not filter_mask is None:
        fftshift[filter_mask != 255] = 0

    f_ishift = np.fft.ifftshift(fftshift)
    return np.abs(np.fft.ifft2(f_ishift))

def create_filter_mask(size, rows, columns):
    ''' Creates a filter mask specified by rows and columns. Specified rows and columns are set to 255, others 0.
    
    Parameters
    ----------
    size : tuple
        Size of resulting filter mask image.
    Returns
    -------
    filter_mask : 2D array
        2D array mask containing 255 and 0 values.
    '''
    if type(size) != tuple:
        raise Exception('Size param must be tuple!')
    
    filter_mask = np.zeros(size, dtype=np.uint8)
    filter_mask[rows] = 255
    filter_mask[:,columns] = 255
    
    return filter_mask

def filter_mag_spec(mag_spec, filter_mask):
    ''' Filters input spektrum using filter_mask image.
    
    Parameters
    ----------
    mag_spec : 2D array
        Image with magnitude spectrum.
    filter_mask : 2D array
        Filter binary mask image containing values to keep (255) and filter out (0).
    Returns
    -------
    result : 2D array
        Vizualization of spectrum after filtering.
    '''        
    result = np.copy(mag_spec)
    result[filter_mask != 255] = 0
    
    return result

In [11]:
def show_images(*imgs, scale=1, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    scale : double
        Scale of shown image window.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    
    See known bug for Mac users
    ---------------------------
    https://gitlab.fit.cvut.cz/bi-svz/bi-svz/issues/13
    """
    def print_xy(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            print('x = %d, y = %d'% (x, y)) 
            
    for i, img in enumerate(imgs, 1):
        h,w = img.shape[:2]
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, int(w * scale), int(h * scale))
        cv2.setMouseCallback(window_name_id, print_xy)
        cv2.moveWindow(window_name_id, (i-1)*int(w * scale), 0)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(0)
        
        if k == ord('q') or k == ord('Q') or k == 27:
            break

    cv2.destroyAllWindows()
    
def plot_images(*imgs, titles=[], channels='bgr', normalize=False, ticks_off=True):
    assert channels.lower() in ['bgr', 'rgb', 'mono'], 'Possible values for channels are: bgr, rgb or mono!'
    
#     f = plt.figure(figsize=(30, 20))
    width_def = 60
    height_def = 60
    
    width = math.ceil(math.sqrt(len(imgs)))
    height = math.ceil(len(imgs) / width)
    
    height_def = height_def / 5 * width
#     print(height_def)
    if height_def > 65:
        height_def = 65
    
    f = plt.figure(figsize=(width_def, height_def))
    
#     print(str(width) + ' , ' + str(height))
    for i, img in enumerate(imgs, 1):
        ax = f.add_subplot(height, width, i)
        if ticks_off:
            ax.axis('off')
        
        if len(titles) != 0:
            if len(imgs) != len(titles):
                print('WARNING titles lenght is not the same as images lenght!')
        
            try:
                ax.set_title(str(titles[i-1]))
            except:
                pass
        
        if channels.lower() == 'mono' or img.ndim==2:
            if normalize:
                norm = Normalize()
            else:
                norm = NoNorm()
            ax.imshow(img, cmap=plt.get_cmap('gray'), norm=norm)
        elif channels.lower() == 'rgb':
            ax.imshow(img)
        else:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [12]:
def ocr(img_bin):
    '''Detects text in the file.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image. White objects on black background.
    Returns
    -------
    Text on image.
    '''
    # Tesseract works with black objects on white background.
    img_bin = negative(img_bin)
    return pytesseract.image_to_string(Image.fromarray(img_bin))

In [13]:
def to_intensity(hue_angle):
    '''Converts color angle in HUE definition into intensity value of brightness image in opencv.
    https://www.docs.opencv.org/trunk/df/d9d/tutorial_py_colorspaces.html
    
    Parameters
    ----------
    hue_angle : int
        Angle in HUE definition (0-359).
    Returns
    -------
    Integer value that represents the same HUE value but in opencv brightness image (0-179).
    '''
    return int(hue_angle * 0.5)

def to_angle(hue_intensity):
    '''Converts hue intensity value of brightness image in opencv into hue angle in HUE definition.
    https://www.docs.opencv.org/trunk/df/d9d/tutorial_py_colorspaces.html
    
    Parameters
    ----------
    hue_intensity : int
        Intensity value of brightness image (0-179).
    Returns
    -------
    Integer value that represents the HUE angle (0-359).
    '''
    return hue_intensity * 2

def to_3_channels(image):
    '''Converts 1 channel image to 3 channels.'''
    if len(image.shape) == 3:
        raise Exception('Image already has 3 channels! Use it on binary or grayscale image only.')
    return cv2.merge([image, image, image])

def logical_and(bin_im, bin_mask):
    return cv2.bitwise_and(bin_im, bin_mask)

def order_points(pts):
    '''Sorts the points based on their x-coordinates.'''
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (bl, tl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (br, tr) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

In [14]:
def polar_warp(img, full_radius=True, inverse=False):
    center = (img.shape[0]/2.0, img.shape[1]/2.0)
    
    if full_radius:
        radius = np.sqrt(((img.shape[0]/2.0)**2.0)+((img.shape[1]/2.0)**2.0))
    else:
        radius = center[0]
    
    method = cv2.WARP_FILL_OUTLIERS
    if inverse: 
        method += cv2.WARP_INVERSE_MAP
    dest = cv2.linearPolar(img, center, radius, method)
    return dest

In [15]:
def warp_to_cartesian(img, full_radius=True):
    return polar_warp(img, full_radius)

def warp_to_polar(img, full_radius=True):
    return polar_warp(img, full_radius, True)

def rotate(img, angle):
    height, width = img.shape[:2]
    image_center = (width/2, height/2)

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    dest = cv2.warpAffine(img, rotation_mat, (bound_w, bound_h))
    return dest

In [16]:
def copy_to(src, dst, mask):
    '''Python alternative to C++/Java OpenCV's Mat.copyTo().
    More: https://docs.opencv.org/trunk/d3/d63/classcv_1_1Mat.html#a626fe5f96d02525e2604d2ad46dd574f'''
    locs = np.where(mask != 0) # Get the non-zero mask locations
    dst[locs[0], locs[1]] = src[locs[0], locs[1]]
    return dst

def midpoint(ptA, ptB):
    '''Returns the midpoint between two input points.'''
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def order_points(pts):
    '''Sorts the points based on their x-coordinates.'''
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (bl, tl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (br, tr) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

def rotate_image(image, angle, image_center=None):
    """ Rotates the input image by specified angle.
    
    Parameters
    ----------
    image : np.ndarray
        Image to be rotated.
    angle : float
        Rotation angle.
    image_center : Optional[tuple(int, int)]
        Center of rotation.
    Returns
    -------
    np.ndarray
        Returns the rotated input image by specified angle.
    """
    if image_center is None:
        image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def draw_rotated_text(img, text, point, angle, text_scale, text_color, text_thickness, units='cm'):
    img_filled = np.full(img.shape, text_color, dtype=np.uint8)
    # create rotated text mask
    text_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    cv2.putText(text_mask, "{:.2f} {}".format(text, units), point, 0, text_scale, (255, 255, 255), text_thickness)
    #angle = -angle + 90
    text_mask = rotate_image(text_mask, -angle, point)
    result = copy_to(img_filled, img.copy(), text_mask)
    return result

def draw_real_sizes(img, rect, width_text, height_text, lbl_size_scale=2, lbl_color=(0, 0, 255), lbl_thickness=8):
    tl, tr, br, bl = order_points(cv2.boxPoints(rect))
    mid_pt_width = midpoint(tl, tr)
    mid_pt_height = midpoint(tr, br)
    
    # bottom-left points where labels are drawn
    pt_label_first =  (int(mid_pt_width[0] - 10), int(mid_pt_width[1] - 10))
    pt_label_second = (int(mid_pt_height[0] + 10), int(mid_pt_height[1]))
        
    result = draw_rotated_text(img, width_text, pt_label_first, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    result = draw_rotated_text(result, height_text, pt_label_second, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    return result

In [17]:
def reindex_image_files(source_dir, output_dir=None):
    """ Reads all images in source_dir and based on they original order, 
    change their filename to be continuous integer (starting from 0). 
    Then, they can be easily read by cv2.VideoCapture. Image format is kept.
    
    Parameters
    ----------
    source_dir : string
        Input images directory that have to be renamed.
    output_dir : Optional[string]
        Output directory for renamed files. If not specified, renaming is done inplace in source_dir.
    Returns
    -------
    None
    """
    input_files = []
    
    for file in os.listdir(source_dir):
        if re.match(r'.*(\.bmp|\.jpg|\.png|\.gif)$', file, re.I):
            input_files.append(os.path.join(source_dir, file))

    if not input_files:
        print('No files were found.')
        return
    
    extension = '.' + input_files[0].split(".")[-1]
    if output_dir is None:
        for i, filename in enumerate(natsorted(input_files)):
            os.rename(filename, os.path.join(source_dir, str(i) +  extension))
        print(f'Files within {source_dir} were renamed, starting from 0{extension} to {i}{extension}.')
    else:
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

        for i, filename in enumerate(natsorted(input_files)):
            shutil.copy(filename, os.path.join(output_dir, str(i) + extension))
            
        print(f'Files from {source_dir} were renamed and saved to {output_dir}, starting from 0{extension} to {i}{extension}.')

def create_file_path(folder, file_name):
    '''Easier defined function to create path for filename inside a folder.
    
    Parameters
    ----------
    folder : string
        Base folder directory in string notation. 
    file_name : string
        File name that should be inside the base folder.
    Returns
    -------
    string
        Path to the newly created file.
    """
    '''
    if not os.path.isdir(folder):
        os.mkdir(folder)
        
    return os.path.join(folder, file_name)

def camera_calib(input_source, chess_shape, output_calib_file=None, img_show_delay=1):
    """ Browses all images found in input_source and on each image tries to find chessboard corners.
    If chessboard corners are found, image corespondences with real world space are added to lists.
    Based on these image-world corespondences, camera calibration is made.
    
    Parameters
    ----------
    input_source : string
        Input source for cv2.VideoCapture (could be camera source or sequence of saved images where format has to be specified).
    chess_shape : tuple
        Number of inner corners per a chessboard row and column.
    output_calib_file : Optional[string]
        Output file where calibration is saved when neccesary.
    img_show_delay : int
        Delay in ms between shown images.
    Returns
    -------
    tuple
        camera matrix and distance coefficients
    """
    cap = cv2.VideoCapture(input_source)

    if not cap.isOpened():
        cv2.destroyAllWindows()
        raise FileNotFoundError('Capture cannot be opened.')

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chess_shape[0] * chess_shape[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chess_shape)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))
            imgpoints.append(corners)

    
    print('Computing camera matrix...')
    rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    if output_calib_file is not None:
        save_camera_calib(output_calib_file, camera_matrix, dist_coefs)

    print('\nRMS:', rms)
    print('Camera matrix:\n', camera_matrix)
    print('Distortion coefficients: ', dist_coefs.ravel())
    
    return camera_matrix, dist_coefs

def correct_frame(frame, camera_matrix, dist_coeffs):
    """Returns undistorted frame."""
    return cv2.undistort(frame, camera_matrix, dist_coeffs)

IDX_CAM_MATRIX = "camera_matrix"
IDX_DIST_COEFFS = "dist_coefs"

def load_camera_calib(input_file):
    """ Loads camera calibration from specified input file.
    
    Parameters
    ----------
    input_file : string
        Input file with calibration data in YAML format.
    Returns
    -------
    tuple(np.array, np.array)
        Returns a tuple where first element is camera matrix array and second element is dist coefficients array. 
        These arrays might be empty if the file isn't found or in correct format.
    """
    try:
        with open(input_file, 'r') as stream:
            data = yaml.load(stream)
            return data[IDX_CAM_MATRIX], data[IDX_DIST_COEFFS]
    except (FileNotFoundError, yaml.YAMLError) as exc:
        print(f'File {input_file} couldn\'t be read.')
        return np.array([]), np.array([])

def save_camera_calib(output_file, camera_matrix, dist_coefs):
    """ Saves camera calibration to specified output file.
    
    Parameters
    ----------
    output_file : string
        Output file used for storing calibration data in YAML format. Parent directory is created if needed.
    Returns
    -------
    None
    """
    data = {IDX_CAM_MATRIX: camera_matrix, IDX_DIST_COEFFS: dist_coefs}
    output_dir = os.path.dirname(output_file)
    
    if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
            
    with open(output_file, "w") as f:
        yaml.dump(data, f)

def create_folder_path(base_folder, new_folder_name):
    """ Creates all neccessary folders in the folder tree structure on computer. 
    
    Parameters
    ----------
    base_folder : string
        Base folder directory in string notation. 
    output_dir : string
        Folder name that should be inside the base folder.
    Returns
    -------
    string
        Path to the newly created folder.
    """
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
            
    path = os.path.join(base_folder, new_folder_name)        
    
    if not os.path.isdir(path):
        os.mkdir(path)      
            
    return path

In [18]:
# Dimensionless descriptors
class ShapeDescriptors:
    def form_factor(area, perimeter):
        return (4 * np.pi * area) / (perimeter * perimeter)
    
    def roundness(area, max_diameter):
        return (4 * area) / (np.pi * max_diameter * max_diameter)
    
    def aspect_ratio(min_diameter, max_diameter):
        return min_diameter / max_diameter;
    
    def convexity(perimeter, convex_perimeter):
        return convex_perimeter / perimeter
    
    def solidity(area, convex_area):
        return area / convex_area
    
    def compactness(area, max_diameter):
        return np.sqrt(4 / np.pi * area) / max_diameter;
        
    def extent(area, bounding_rectangle_area):
        return area / bounding_rectangle_area;

# Špičatost
def form_factor(bin_im):
    _, _, conts = find_contours(bin_im)
    return ShapeDescriptors.form_factor(cv2.contourArea(conts[0]), cv2.arcLength(conts[0], True))

# Kulatost
def roundness(bin_im):
    _, _, conts = find_contours(bin_im)
    area = cv2.contourArea(conts[0])
    _,radius = cv2.minEnclosingCircle(conts[0])
    r = ShapeDescriptors.roundness(area, 2*radius)
    if r > 1: r = 1
    return r

# Poměr stran
def aspect_ratio(bin_im):
    _, _, conts = find_contours(bin_im)
    dims = cv2.minAreaRect(conts[0])[1]
    min_diameter = min(dims)
    max_diameter = max(dims)
    return ShapeDescriptors.aspect_ratio(min_diameter, max_diameter)
    
# Konvexita, vypouklost
def convexity(bin_im):
    _, _, conts = find_contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    per = cv2.arcLength(conts[0], True)
    conv_per = cv2.arcLength(hull, True)
    r = ShapeDescriptors.convexity(per, conv_per)
    if r > 1: r = 1
    return r 

# Plnost, celistvost
def solidity(bin_im):
    _, _, conts = find_contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    area = cv2.contourArea(conts[0])
    conv_area = cv2.contourArea(hull)
    r = ShapeDescriptors.solidity(area, conv_area)
    if r > 1: r = 1
    return r 
    
# Kompaktnost, hutnost
def compactness(bin_im):
    _, _, conts = find_contours(bin_im)
    area = cv2.contourArea(conts[0])
    max_diameter = max(cv2.minAreaRect(conts[0])[1])
    r = ShapeDescriptors.compactness(area, max_diameter)
    if r > 1: r = 1
    return r 
    
# Dosah, rozměrnost
def extent(bin_im):
    _, _, conts = find_contours(bin_im)
    area = cv2.contourArea(conts[0])
    w, h = cv2.minAreaRect(conts[0])[1]
    return ShapeDescriptors.extent(area, w*h)

In [19]:
def calibrate_camera(grabbed_images_folder_path, chess_shape):
    reindex_image_files(grabbed_images_folder_path)
    images_format = '%01d.png'
    calibration_file_name = 'config.yaml'
    output_calib_file_path = create_file_path(grabbed_images_folder_path, calibration_file_name)
    return camera_calib(create_file_path(grabbed_images_folder_path, images_format), chess_shape=chess_shape,output_calib_file=output_calib_file_path)

def fix_camera_picture(image, camera_matrix, dist_coefs):
    return correct_frame(image, camera_matrix, dist_coefs)

In [20]:
def plot_image_histogram(image, brg = False):
    color = ('r','g','b')
    if brg:
        color = ('b','r','g')
    for i,col in enumerate(color):
        histr = cv2.calcHist([img],[i],None,[256],[0,256])
        plt.plot(histr,color = col)
        plt.xlim([0,256])
        plt.ylim([0,8000])
    plt.show()

In [21]:
def plot_colored_square(size=30, rgb_color=(0, 0, 0)):
    image = np.zeros((size, size, 3), np.uint8)
    color = tuple(reversed(rgb_color))
    image[:] = color

    return image

In [22]:
def highlight_points(image, points, size):
    box_points_drawn = image.copy()
    for p in points:
        cv2.circle(box_points_drawn,(p[0],p[1]), size, (0,0,255), -1)
    return box_points_drawn

In [23]:
# returns hue values min and max for segmentation. Select starting point in image start=(num1, num2) and it returns lower and upper bound.

def find_hsv_minmax(hsv_image, start, scope):
    low = [255, 255, 255]
    high = [0, 0, 0]
    for j in range(start[0] - int(scope/2), start[0] + int(scope/2 + scope%2)):
        for i in range(start[1] - int(scope/2), start[1] + int(scope/2 + scope%2)):
            if low[0] > hsv_image[i][j][0]:
                low[0] = hsv_image[i][j][0]
            if low[1] > hsv_image[i][j][1]:
                low[1] = hsv_image[i][j][1]
            if low[2] > hsv_image[i][j][2]:
                low[2] = hsv_image[i][j][2]
            
            if high[0] < hsv_image[i][j][0]:
                high[0] = hsv_image[i][j][0]
            if high[1] < hsv_image[i][j][1]:
                high[1] = hsv_image[i][j][1]
            if high[2] < hsv_image[i][j][2]:
                high[2] = hsv_image[i][j][2]
    return low, high

# lower_bound, upper_bound = find_hsv_bounds(image, indices, 5)
def find_hsv_bounds(image, points, scope, point_area=5):
    plot_images(highlight_points(image, points, point_area))
    low = [255, 255, 255]
    high = [0, 0, 0]
    hsv_image = to_hsv(image)
    for point in points:
        l, h = find_hsv_minmax(hsv_image, point, scope)
        if low[0] > l[0]:
            low[0] = l[0]
        if low[1] > l[1]:
            low[1] = l[1]
        if low[2] > l[2]:
            low[2] = l[2]

        if high[0] < h[0]:
            high[0] = h[0]
        if high[1] < h[1]:
            high[1] = h[1]
        if high[2] < h[2]:
            high[2] = h[2]
    print(f'lower bound: {low}')
    print(f'upper bound: {high}')
    return tuple(low), tuple(high)

In [24]:
# mask = create_hsv_mask(image, lower_bound, upper_bound, tolerance=10)
def create_hsv_mask(image, low, high, tolerance=10):
    l1 = int(max(0, low[0]-tolerance))
    l2 = int(max(0, low[1]-tolerance))
    l3 = int(max(0, low[2]-tolerance))

    u1 = int(min(255, high[0]+tolerance))
    u2 = int(min(255, high[1]+tolerance))
    u3 = int(min(255, high[2]+tolerance))
    img_patterns_color = to_hsv(image)
    return cv2.inRange(img_patterns_color, (l1, l2, l3),(u1, u2, u3))

In [25]:
# contours = get_contours(mask, number_of_contours=5, min_area=1)
def get_contours(mask, number_of_contours=1, min_area=0, max_area=-1):
    contours, _  = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    contours = [contour for contour in contours if cv2.contourArea(contour) >= min_area]
    if max_area > min_area:
        contours = [contour for contour in contours if cv2.contourArea(contour) <= max_area]
    print(f'Found {len(contours)} contour(s). Returning sorted from largest: {number_of_contours}')
    return contours[:number_of_contours]

In [26]:
# mask = get_image_from_contour([contour[0]], image, False)[0]
def get_image_from_contour(contours, image, cut=False):
    blank = np.zeros_like(image)
    contour_image = cv2.drawContours(blank, contours, -1, (255, 255, 255), cv2.FILLED)
    if not cut:
        return [contour_image]
    contour_images = []
    for contour in contours:
        blank = np.zeros_like(image)
        contour_image = to_gray(cv2.drawContours(blank, [contour], -1, (255, 255, 255), cv2.FILLED))
        contour_images.append(crop_by_bounding_rect(contour_image))
    return contour_images

In [27]:
# mask_erode = erode(mask, get_square_kernel(40))
def erode(mask, kernel):
    return cv2.erode(mask, kernel, iterations=1)

# mask_dilate = dilate(mask, get_square_kernel(65))
def dilate(mask, kernel):
    return cv2.dilate(mask, kernel, iterations=1)

def get_square_kernel(size):
    return np.ones((size,size), np.uint8)

In [28]:
# plot_images(highlight_contour(contours, image))
def highlight_contour(contours, image):
    contour_drawn = cv2.drawContours(image.copy(), contours, -1, color=(255, 0, 0), thickness=5)
    return contour_drawn

In [29]:
def get_rect_size_from_contour(contour, scale):
    rect = cv2.minAreaRect(contour)
    if abs(rect[2]) < 45:
        shape_height, shape_width = rect[1]   
        return shape_width*scale, shape_height*scale
    else: 
        shape_width, shape_height = rect[1] 
        return shape_width*scale, shape_height*scale

# plot_images(get_rect_sizes(image, contours, 0.017167381974248927))
def get_rect_sizes(image, contours, real_image_ratio):
    image_with_measurement = image.copy()
    for contour in contours:
        rect = cv2.minAreaRect(contour)
        if abs(rect[2]) < 45:
            shape_height, shape_width = rect[1]   
        else: 
            shape_width, shape_height = rect[1] 

        real_width = (shape_width*real_image_ratio) 
        real_height = (shape_height*real_image_ratio) 

        cv2.drawContours(image_with_measurement, [contour], -1, color=(255, 0, 0 ), thickness=5)
        image_with_measurement = draw_real_sizes(
            image_with_measurement,
            rect,
            real_height,
            real_width,
            lbl_size_scale=.7,
            lbl_color=(0, 0, 255),
            lbl_thickness=1
        )
    return image_with_measurement

In [30]:
def draw_line(original_image, a_pos, b_pos):
    image = copy.deepcopy(original_image)
    a_pos = (int(a_pos[0]), int(a_pos[1]))
    b_pos = (int(b_pos[0]), int(b_pos[1]))
    cv2.line(image, a_pos, b_pos, (0, 0, 255), 5)
    return image

# contour_area_image = draw_number(contour_area_image, area, get_center(contours[2]), 'cm^2')
def draw_number(original_image, number, position, units='cm'):
    image = copy.deepcopy(original_image)
    position = (int(position[0]), int(position[1]))
    image = draw_rotated_text(
        image, 
        number, 
        position, 
        0, 
        1, 
        (255, 0, 0), 
        2,
        units
    )
    return image

# dist_cont_image, distance, point_a, point_b = get_contours_distances(image, [contours[0], contours[1]], 0.017167381974248927)
def get_contours_distances(original_image, contours, ratio):
    image = copy.deepcopy(original_image)
    indices = copy.deepcopy(contours)
    contour_map = list(itertools.combinations(indices , 2))

    for comb in contour_map:
        dist = None
        a_pos = []
        b_pos = []
        for a in comb[0]:
            for b in comb[1]:
                x1 = a[0][0]
                y1 = a[0][1]
                x2 = b[0][0]
                y2 = b[0][1]
                dx = x2 - x1
                dy = y2 - y1
                current_dist = (dx * dx + dy * dy) ** 0.5
                if dist is not None:
                    if dist > current_dist:
                        dist = current_dist
                        a_pos = a[0]
                        b_pos = b[0]
                else:
                    dist = current_dist
                    a_pos = a[0]
                    b_pos = b[0]
        image = draw_line(image, a_pos, b_pos)
        image = draw_number(
            image, 
            dist*ratio, 
            (
                int((a_pos[0] + b_pos[0])/2), 
                int((a_pos[1] + b_pos[1])/2)
            )
        )
    return image, dist*ratio, a_pos, b_pos

In [31]:
# get_contour_area(contours[2], image, 0.0002889)
def get_contour_area(contour, image, scale):
    area = cv2.contourArea(contour)
    return area*scale

In [32]:
# contour = get_contour_containing_point(contours, image, [(1151, 321), (300, 380)])
def get_contour_containing_point(contours, image, points):
    found_contours = []
    plot_images(highlight_points(image, points, 5))
    for point in points:
        point = [int(point[0]), int(point[1])]
        for contour in contours:
            mask = get_image_from_contour([contour], image, False)[0]
            if (mask[point[1]][point[0]] != [0, 0, 0]).all():
                found_contours.append(contour)
    return found_contours

In [33]:
# get_contour_similarity_difference(contour_a, contour_b, key_a, key_b)
def get_contour_similarity_difference(contour_a, contour_b, image_a, image_b):
    functions = [form_factor, roundness, aspect_ratio, convexity, solidity, compactness, extent]
    descriptions_a = []
    descriptions_b = []
    image_a = get_image_from_contour([contour_a], image_a, cut=True)[0]
    image_a = to_gray(image_a)
    image_b = get_image_from_contour([contour_b], image_b, cut=True)[0]
    image_b = to_gray(image_b)
    for func in functions:
        descriptions_a.append(func(image_a))
        descriptions_b.append(func(image_b))
    difference = np.linalg.norm(np.array(descriptions_a) - np.array(descriptions_b))
    return difference

# sorted_contours = sort_contours_by_similarity(contours[0], contours, image)
def sort_contours_by_similarity(ref_contour, contours, image):
    contour_list = []
    cont = []
    diff = []
    for contour in contours:
        similarity = get_contour_similarity_difference(ref_contour, contour, image, image)
        contour_list.append([similarity, contour])
    return [(k, v) for k, v in sorted(contour_list, key=lambda item: item[0])]

In [34]:
# mask_union(mask_erode, mask_dilate)
def mask_union(mask_a, mask_b):
    return cv2.bitwise_or(mask_a, mask_b)

# mask_intersect(mask_erode, mask_dilate)
def mask_intersect(mask_a, mask_b):
    return cv2.bitwise_and(mask_a, mask_b)